# SW-13-GraphRAG

**Navigation** : [<< 12-KnowledgeGraphs](SW-12-KnowledgeGraphs.ipynb) | [Index](README.md)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Comprendre le paradigme GraphRAG et ses avantages par rapport au RAG classique
2. Combiner des graphes de connaissances (KG) avec des LLMs pour ameliorer la generation augmentee
3. Implementer un pipeline KG-enhanced retrieval complet
4. Evaluer les compromis entre RAG traditionnel et GraphRAG

### Concepts cles

| Concept | Description |
|---------|-------------|
| RAG | Retrieval-Augmented Generation : enrichir un LLM avec des documents retrouves |
| GraphRAG | RAG augmente par la structure d'un graphe de connaissances |
| KG (Knowledge Graph) | Graphe structure reliant des entites par des relations typees |
| Extraction d'entites | Identification automatique de personnes, lieux, organisations dans du texte |
| Subgraph retrieval | Extraction d'un sous-graphe pertinent autour d'entites identifiees |

### Prerequis
- SW-8 (rdflib et Python RDF)
- SW-12 (Graphes de connaissances)
- Cle API OpenAI ou Anthropic (optionnelle : le notebook fonctionne en mode degrade sans cle)

### Duree estimee : 50 minutes

---

## Installation et configuration

Installons les dependances necessaires. Les packages `openai` et `anthropic` sont optionnels : le notebook fonctionnera en mode simule si aucune cle API n'est configuree.

In [ ]:
!pip install -q rdflib openai anthropic networkx python-dotenv

### Configuration de l'environnement

Nous chargeons les variables d'environnement depuis un fichier `.env` (s'il existe) et detectons la disponibilite des cles API. Le notebook adapte son comportement automatiquement.

In [ ]:
import os
import re
import json
from collections import defaultdict

# Chargement .env (optionnel)
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

import rdflib
from rdflib import Graph, Namespace, Literal, URIRef, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD, FOAF
import networkx as nx

# Detection des cles API
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_KEY = os.getenv("ANTHROPIC_API_KEY")
HAS_LLM = bool(OPENAI_KEY or ANTHROPIC_KEY)

if HAS_LLM:
    provider = "OpenAI" if OPENAI_KEY else "Anthropic"
    print(f"Cle API detectee : {provider}")
    print("Les sections LLM seront executees avec de vrais appels API.")
else:
    print("Aucune cle API detectee (OPENAI_API_KEY / ANTHROPIC_API_KEY).")
    print("Les sections LLM afficheront des resultats simules.")
    print("Pour activer les appels LLM, creez un fichier .env avec votre cle.")

print(f"\nrdflib version : {rdflib.__version__}")

---

## 1. Du RAG au GraphRAG

### Le RAG classique : rappel

Le **Retrieval-Augmented Generation (RAG)** est un paradigme qui enrichit la generation par LLM avec des informations retrouvees dans une base documentaire. Le pipeline classique est :

```
Question utilisateur
       |
       v
[Embedding de la question]
       |
       v
[Recherche vectorielle dans la base]
       |
       v
[Top-K documents pertinents]
       |
       v
[Contexte + Question -> LLM]
       |
       v
Reponse generee
```

### Les limitations du RAG classique

| Limitation | Description | Exemple |
|-----------|-------------|----------|
| Pas de structure | Les chunks sont des blocs de texte plat | Relations entre entites perdues |
| Pas de relations | La similarite vectorielle ne capture pas les liens | "Qui sont les allies de X ?" echoue |
| Requetes globales | Questions sur l'ensemble du corpus mal gerees | "Quel est le theme principal ?" |
| Contexte fragmente | Les chunks decoupent l'information | Un fait reparti sur 3 paragraphes |
| Pas de raisonnement | Aucune inference logique sur les donnees | Transitivite, heritage ignorees |

### GraphRAG : ajouter la structure

**GraphRAG** enrichit le pipeline RAG en ajoutant un **graphe de connaissances** comme source de contexte structuree. Au lieu de chercher uniquement des passages textuels similaires, on interroge un KG pour obtenir des faits structures et des relations.

```
Question utilisateur
       |
       v
[Detection d'entites dans la question]
       |                    |
       v                    v
[Recherche vectorielle]  [Requete KG (SPARQL)]
       |                    |
       v                    v
[Passages textuels]  [Sous-graphe structure]
       \                  /
        v                v
     [Contexte combine -> LLM]
              |
              v
       Reponse enrichie
```

### Microsoft GraphRAG (2024)

En 2024, Microsoft Research a publie **GraphRAG**, une approche qui construit automatiquement un graphe de connaissances a partir d'un corpus de texte. Les etapes cles :

1. **Extraction d'entites** : un LLM identifie les entites et relations dans chaque document
2. **Construction du graphe** : les entites et relations forment un graphe global
3. **Detection de communautes** : l'algorithme de Leiden identifie des clusters thematiques
4. **Resumes de communautes** : chaque cluster est resume par un LLM
5. **Requete** : les questions utilisent a la fois le graphe et les resumes

### Comparaison RAG vs GraphRAG

| Critere | RAG classique | GraphRAG |
|---------|---------------|----------|
| Source de contexte | Chunks textuels | Chunks + sous-graphe KG |
| Structure des donnees | Plat (vecteurs) | Structure (triplets RDF) |
| Requetes locales | Bon | Bon |
| Requetes globales | Faible | Fort (resumes communautes) |
| Relations multi-sauts | Faible | Fort (traversee de graphe) |
| Cout de construction | Faible (embedding) | Eleve (extraction LLM) |
| Cout de requete | Faible | Moyen (SPARQL + LLM) |
| Hallucinations | Moyen | Reduit (faits verifiables) |
| Mise a jour | Facile (re-embed) | Moyenne (re-extraction) |

---

## 2. Construction d'un graphe de connaissances a partir de texte

La premiere etape d'un pipeline GraphRAG est de construire un KG a partir de texte brut. Nous allons explorer deux approches :
1. **Extraction par regles** (regex, patterns) : sans API, deterministe
2. **Extraction par LLM** : plus robuste, necessite une cle API

### Texte source

Nous utiliserons un passage sur l'histoire de France comme texte d'exemple pour l'extraction.

In [ ]:
# Texte source pour l'extraction d'entites et de relations
SAMPLE_TEXT = """
Napoleon Bonaparte est ne en 1769 a Ajaccio, en Corse. Il est devenu empereur
des Francais en 1804 apres avoir mene plusieurs campagnes militaires victorieuses.
Napoleon a fonde la Banque de France en 1800 et a promulgue le Code civil en 1804.
Il a epouse Josephine de Beauharnais en 1796, puis Marie-Louise d'Autriche en 1810.
Sa defaite a Waterloo en 1815 face au duc de Wellington a mis fin a son regne.
Il a ete exile a Sainte-Helene, une ile britannique dans l'Atlantique Sud,
ou il est mort en 1821. Son heritage inclut le Code Napoleon, qui a influence
le droit civil de nombreux pays europeens.
"""

print("Texte source charge.")
print(f"Longueur : {len(SAMPLE_TEXT)} caracteres, {len(SAMPLE_TEXT.split())} mots")

### 2.1 Extraction par regles (sans API)

Cette approche utilise des expressions regulieres et des patterns linguistiques pour identifier des entites et relations. Elle est limitee mais illustre le principe d'extraction de triplets.

In [ ]:
def extract_entities_by_rules(text):
    """Extraction d'entites par regles (regex)."""
    entities = {
        "persons": [],
        "places": [],
        "dates": [],
        "organizations": []
    }

    # Dates : annees a 4 chiffres
    entities["dates"] = list(set(re.findall(r'\b(1[0-9]{3}|20[0-2][0-9])\b', text)))

    # Personnes : noms propres composes (prenom + nom)
    person_patterns = [
        r'(Napoleon Bonaparte)',
        r'(Napoleon)',
        r'(Josephine de Beauharnais)',
        r'(Marie-Louise d\'Autriche)',
        r'(duc de Wellington)',
    ]
    for pattern in person_patterns:
        matches = re.findall(pattern, text)
        entities["persons"].extend(matches)
    entities["persons"] = list(set(entities["persons"]))

    # Lieux : detection basee sur des indices contextuels
    place_patterns = [
        r'a (Ajaccio)',
        r'en (Corse)',
        r'a (Waterloo)',
        r'a (Sainte-Helene)',
        r'l\'(Atlantique Sud)',
    ]
    for pattern in place_patterns:
        matches = re.findall(pattern, text)
        entities["places"].extend(matches)
    entities["places"] = list(set(entities["places"]))

    # Organisations
    org_patterns = [
        r'(Banque de France)',
        r'(Code civil)',
        r'(Code Napoleon)',
    ]
    for pattern in org_patterns:
        matches = re.findall(pattern, text)
        entities["organizations"].extend(matches)
    entities["organizations"] = list(set(entities["organizations"]))

    return entities


entities = extract_entities_by_rules(SAMPLE_TEXT)

print("Entites extraites par regles :")
print(f"  Personnes     : {entities['persons']}")
print(f"  Lieux         : {entities['places']}")
print(f"  Dates         : {sorted(entities['dates'])}")
print(f"  Organisations : {entities['organizations']}")

### Interpretation : extraction par regles

L'extraction par regles a identifie les entites les plus evidentes du texte.

| Type | Nombre | Exemples |
|------|--------|----------|
| Personnes | ~4-5 | Napoleon Bonaparte, Josephine de Beauharnais |
| Lieux | ~4-5 | Ajaccio, Corse, Waterloo, Sainte-Helene |
| Dates | ~6-7 | 1769, 1796, 1800, 1804, 1810, 1815, 1821 |
| Organisations | ~2-3 | Banque de France, Code civil |

**Limites de cette approche** :
- Les patterns sont specifiques au texte (non generalisables)
- Les entites ambigues ne sont pas desambiguisees
- Les relations entre entites ne sont pas extraites
- Un nouveau texte necessite de nouveaux patterns

> **Note technique** : En production, on utiliserait des outils de NER (Named Entity Recognition) comme spaCy ou des modeles transformer pour une extraction plus robuste.

### 2.2 Generation de triplets a partir du texte

Nous allons maintenant extraire des **relations** entre les entites pour former des triplets (sujet, predicat, objet). Commençons par une approche par regles.

In [ ]:
def extract_triples_by_rules(text):
    """Extraction de triplets par patterns linguistiques."""
    triples = []

    # Pattern : X est ne en ANNEE a LIEU
    m = re.search(r'(Napoleon Bonaparte) est ne en (\d{4}) a (\w+)', text)
    if m:
        triples.append((m.group(1), "ne_en_annee", m.group(2)))
        triples.append((m.group(1), "ne_a", m.group(3)))

    # Pattern : X est devenu Y en ANNEE
    m = re.search(r'(\w+) est devenu (empereur[^.]*) en (\d{4})', text)
    if m:
        triples.append((m.group(1).strip(), "titre", "empereur des Francais"))
        triples.append((m.group(1).strip(), "debut_regne", m.group(3)))

    # Pattern : X a fonde Y en ANNEE
    for m in re.finditer(r'(Napoleon|Il) a fonde (la [^.]+?) en (\d{4})', text):
        triples.append(("Napoleon", "a_fonde", m.group(2).strip()))
        triples.append((m.group(2).strip(), "date_creation", m.group(3)))

    # Pattern : X a promulgue Y en ANNEE
    for m in re.finditer(r'(?:Napoleon|[Ii]l) a promulgue (le [^.]+?) en (\d{4})', text):
        triples.append(("Napoleon", "a_promulgue", m.group(1).strip()))

    # Pattern : X a epouse Y en ANNEE
    for m in re.finditer(r'(?:Il|Napoleon) a epouse (\w[^,]+?) en (\d{4})', text):
        triples.append(("Napoleon", "a_epouse", m.group(1).strip()))
        triples.append(("Napoleon", "mariage_annee", m.group(2)))

    # Pattern : defaite a LIEU en ANNEE face a PERSONNE
    m = re.search(r'defaite a (\w+) en (\d{4}) face au (\w+ de \w+)', text)
    if m:
        triples.append(("Napoleon", "defaite_a", m.group(1)))
        triples.append(("Bataille de " + m.group(1), "date", m.group(2)))
        triples.append(("Napoleon", "vaincu_par", m.group(3)))

    # Pattern : exile a LIEU
    m = re.search(r'exile a (Sainte-Helene)', text)
    if m:
        triples.append(("Napoleon", "exile_a", m.group(1)))

    # Pattern : mort en ANNEE
    m = re.search(r'mort en (\d{4})', text)
    if m:
        triples.append(("Napoleon", "mort_en", m.group(1)))

    return triples


triples = extract_triples_by_rules(SAMPLE_TEXT)

print(f"Triplets extraits : {len(triples)}\n")
print(f"{'Sujet':<25} {'Predicat':<20} {'Objet'}")
print("-" * 75)
for s, p, o in triples:
    print(f"{s:<25} {p:<20} {o}")

### Interpretation : triplets extraits

L'extraction par regles a produit une dizaine de triplets structures. Chaque triplet represente un fait atomique :

| Categorie | Exemple de triplet | Fiabilite |
|-----------|-------------------|------------|
| Biographie | (Napoleon, ne_a, Ajaccio) | Haute |
| Chronologie | (Napoleon, ne_en_annee, 1769) | Haute |
| Relations | (Napoleon, a_epouse, Josephine de Beauharnais) | Haute |
| Evenements | (Napoleon, defaite_a, Waterloo) | Haute |
| Realisations | (Napoleon, a_fonde, la Banque de France) | Haute |

**Points cles** :
1. Les triplets sont des faits **atomiques** et **verifiables**
2. Les predicats sont normalises (conventions de nommage)
3. L'approche par regles est **precise** mais **peu generalisable**

### 2.3 Extraction par LLM (optionnelle)

Un LLM peut extraire des triplets de maniere beaucoup plus robuste et generalisable. Cette section necessite une cle API. Sans cle, un resultat simule sera affiche.

In [ ]:
def extract_triples_llm(text):
    """Extraction de triplets via LLM (OpenAI ou Anthropic)."""
    prompt = f"""Extrait tous les triplets (sujet, predicat, objet) du texte suivant.
Retourne le resultat en JSON : une liste de dictionnaires avec les cles "subject", "predicate", "object".
Utilise des predicats normalises en snake_case.

Texte :
{text}

Retourne UNIQUEMENT le JSON, sans commentaire."""

    try:
        if OPENAI_KEY:
            from openai import OpenAI
            client = OpenAI(api_key=OPENAI_KEY)
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.0
            )
            raw = response.choices[0].message.content
        elif ANTHROPIC_KEY:
            from anthropic import Anthropic
            client = Anthropic(api_key=ANTHROPIC_KEY)
            response = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=2000,
                messages=[{"role": "user", "content": prompt}]
            )
            raw = response.content[0].text
        else:
            return None

        # Nettoyage du JSON (enlever les balises markdown)
        raw = raw.strip()
        if raw.startswith("```"):
            raw = re.sub(r'^```(?:json)?\n?', '', raw)
            raw = re.sub(r'\n?```$', '', raw)
        return json.loads(raw)

    except Exception as e:
        print(f"Erreur lors de l'appel LLM : {e}")
        return None


# Resultat simule (utilise si pas de cle API)
SIMULATED_LLM_TRIPLES = [
    {"subject": "Napoleon Bonaparte", "predicate": "born_in_year", "object": "1769"},
    {"subject": "Napoleon Bonaparte", "predicate": "born_in_place", "object": "Ajaccio"},
    {"subject": "Ajaccio", "predicate": "located_in", "object": "Corse"},
    {"subject": "Napoleon Bonaparte", "predicate": "became", "object": "Empereur des Francais"},
    {"subject": "Napoleon Bonaparte", "predicate": "became_emperor_year", "object": "1804"},
    {"subject": "Napoleon Bonaparte", "predicate": "founded", "object": "Banque de France"},
    {"subject": "Banque de France", "predicate": "founded_year", "object": "1800"},
    {"subject": "Napoleon Bonaparte", "predicate": "promulgated", "object": "Code civil"},
    {"subject": "Code civil", "predicate": "promulgated_year", "object": "1804"},
    {"subject": "Napoleon Bonaparte", "predicate": "married", "object": "Josephine de Beauharnais"},
    {"subject": "Napoleon Bonaparte", "predicate": "marriage_year", "object": "1796"},
    {"subject": "Napoleon Bonaparte", "predicate": "married", "object": "Marie-Louise d'Autriche"},
    {"subject": "Napoleon Bonaparte", "predicate": "marriage_year", "object": "1810"},
    {"subject": "Napoleon Bonaparte", "predicate": "defeated_at", "object": "Waterloo"},
    {"subject": "Bataille de Waterloo", "predicate": "year", "object": "1815"},
    {"subject": "Napoleon Bonaparte", "predicate": "defeated_by", "object": "Duc de Wellington"},
    {"subject": "Napoleon Bonaparte", "predicate": "exiled_to", "object": "Sainte-Helene"},
    {"subject": "Sainte-Helene", "predicate": "type", "object": "Ile britannique"},
    {"subject": "Sainte-Helene", "predicate": "located_in", "object": "Atlantique Sud"},
    {"subject": "Napoleon Bonaparte", "predicate": "died_year", "object": "1821"},
    {"subject": "Napoleon Bonaparte", "predicate": "legacy", "object": "Code Napoleon"},
    {"subject": "Code Napoleon", "predicate": "influenced", "object": "droit civil europeen"}
]

if HAS_LLM:
    llm_triples = extract_triples_llm(SAMPLE_TEXT)
    if llm_triples is None:
        print("Appel LLM echoue. Utilisation des resultats simules.")
        llm_triples = SIMULATED_LLM_TRIPLES
    else:
        print(f"Triplets extraits par LLM : {len(llm_triples)}")
else:
    print("Mode simule (pas de cle API).")
    llm_triples = SIMULATED_LLM_TRIPLES
    print(f"Triplets simules : {len(llm_triples)}")

print(f"\n{'Sujet':<28} {'Predicat':<22} {'Objet'}")
print("-" * 80)
for t in llm_triples:
    print(f"{t['subject']:<28} {t['predicate']:<22} {t['object']}")

### Interpretation : comparaison regles vs LLM

| Critere | Extraction par regles | Extraction par LLM |
|---------|----------------------|---------------------|
| Nombre de triplets | ~10-12 | ~20-25 |
| Generalisabilite | Faible (patterns specifiques) | Forte (tout texte) |
| Precision | Haute (patterns exacts) | Haute (mais hallucinations possibles) |
| Rappel | Faible (beaucoup de faits manques) | Eleve |
| Cout | Gratuit | Payant (tokens API) |
| Reproductibilite | Parfaite | Variable (temperature) |

Le LLM a extrait des relations plus riches, incluant des faits implicites (ex: Sainte-Helene est une ile britannique) que l'approche par regles aurait manques.

> **Point cle** : En pratique, on combine souvent les deux approches : regles pour les patterns connus et fiables, LLM pour couvrir le reste.

---

## 3. Retrieval augmente par graphe de connaissances

Nous allons maintenant construire un veritable graphe RDF a partir des triplets extraits, puis l'utiliser pour du retrieval structure.

### 3.1 Construction du KG en RDF

Convertissons les triplets extraits en un graphe RDF avec rdflib. Nous utilisons un namespace dedie pour notre domaine.

In [ ]:
# Namespace pour notre domaine
EX = Namespace("http://example.org/history/")
REL = Namespace("http://example.org/relation/")


def name_to_uri(name):
    """Convertit un nom en URI valide."""
    slug = name.strip().replace(" ", "_").replace("'", "").replace("'", "")
    slug = re.sub(r'[^a-zA-Z0-9_-]', '', slug)
    return EX[slug]


def build_kg_from_triples(triple_list):
    """Construit un graphe RDF a partir d'une liste de triplets."""
    g = Graph()
    g.bind("ex", EX)
    g.bind("rel", REL)
    g.bind("rdfs", RDFS)

    for t in triple_list:
        subj = name_to_uri(t["subject"])
        pred = REL[t["predicate"]]
        obj_val = t["object"]

        # Si l'objet ressemble a une annee, on en fait un literal
        if re.match(r'^\d{4}$', obj_val):
            obj = Literal(int(obj_val), datatype=XSD.gYear)
        else:
            # On cree un noeud URI pour les entites, un literal pour les descriptions
            if len(obj_val.split()) <= 4 and obj_val[0].isupper():
                obj = name_to_uri(obj_val)
                # Ajouter un label RDFS
                g.add((obj, RDFS.label, Literal(obj_val, lang="fr")))
            else:
                obj = Literal(obj_val, lang="fr")

        g.add((subj, pred, obj))
        # Label pour le sujet
        g.add((subj, RDFS.label, Literal(t["subject"], lang="fr")))

    return g


kg = build_kg_from_triples(llm_triples)

print(f"Graphe de connaissances construit :")
print(f"  Nombre de triplets : {len(kg)}")
print(f"  Sujets uniques     : {len(set(s for s, _, _ in kg))}")
print(f"  Predicats uniques  : {len(set(p for _, p, _ in kg))}")
print(f"\nExtrait du graphe (10 premiers triplets) :")
for i, (s, p, o) in enumerate(kg):
    if i >= 10:
        print("  ...")
        break
    s_label = s.split('/')[-1].replace('_', ' ')
    p_label = p.split('/')[-1]
    o_label = str(o).split('/')[-1].replace('_', ' ') if isinstance(o, URIRef) else str(o)
    print(f"  ({s_label}, {p_label}, {o_label})")

### Interpretation : KG construit

Le graphe RDF contient desormais les faits structures extraits du texte. Chaque entite est identifiee par une URI unique, et les relations sont typees.

**Points cles** :
1. Les entites nommees (personnes, lieux) sont des `URIRef` pour permettre les liens
2. Les dates sont des `Literal` types (`xsd:gYear`) pour les requetes numeriques
3. Les labels RDFS ajoutent des noms lisibles aux URIs
4. Le graphe est interrogeable via SPARQL

### 3.2 Requetes SPARQL comme contexte

L'avantage principal du KG est de pouvoir repondre a des questions structurees via SPARQL, ce que la recherche vectorielle ne peut pas faire.

In [ ]:
# Requete 1 : Tous les faits sur Napoleon
query_napoleon = """
PREFIX ex: <http://example.org/history/>
PREFIX rel: <http://example.org/relation/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?predicate ?object ?obj_label
WHERE {
    ex:Napoleon_Bonaparte ?predicate ?object .
    FILTER(?predicate != rdfs:label)
    OPTIONAL { ?object rdfs:label ?obj_label }
}
ORDER BY ?predicate
"""

print("Requete : Tous les faits sur Napoleon Bonaparte")
print("=" * 60)
results = kg.query(query_napoleon)
for row in results:
    pred = str(row.predicate).split('/')[-1]
    obj = str(row.obj_label) if row.obj_label else str(row.object).split('/')[-1].replace('_', ' ')
    print(f"  {pred:<25} {obj}")

print(f"\nTotal : {len(results)} faits retrouves")

Testons une deuxieme requete, de type multi-sauts : quels lieux sont lies a Napoleon ?

In [ ]:
# Requete 2 : Lieux lies a Napoleon (multi-sauts)
query_places = """
PREFIX ex: <http://example.org/history/>
PREFIX rel: <http://example.org/relation/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?relation ?place ?place_label ?detail_pred ?detail_obj
WHERE {
    ex:Napoleon_Bonaparte ?relation ?place .
    FILTER(CONTAINS(STR(?relation), "born_in_place") ||
           CONTAINS(STR(?relation), "defeated_at") ||
           CONTAINS(STR(?relation), "exiled_to") ||
           CONTAINS(STR(?relation), "ne_a") ||
           CONTAINS(STR(?relation), "defaite_a") ||
           CONTAINS(STR(?relation), "exile_a"))
    OPTIONAL { ?place rdfs:label ?place_label }
    OPTIONAL {
        ?place ?detail_pred ?detail_obj .
        FILTER(?detail_pred != rdfs:label)
    }
}
"""

print("Requete : Lieux lies a Napoleon (avec details)")
print("=" * 60)
results = kg.query(query_places)
for row in results:
    rel = str(row.relation).split('/')[-1]
    place = str(row.place_label) if row.place_label else str(row.place).split('/')[-1].replace('_', ' ')
    detail = ""
    if row.detail_pred:
        dp = str(row.detail_pred).split('/')[-1]
        do = str(row.detail_obj).split('/')[-1].replace('_', ' ')
        detail = f" -> {dp}: {do}"
    print(f"  {rel:<20} {place}{detail}")

print(f"\nTotal : {len(results)} resultats")

### Interpretation : SPARQL pour le retrieval

Les requetes SPARQL offrent des avantages decisifs par rapport a la recherche vectorielle :

| Capacite | Recherche vectorielle | SPARQL sur KG |
|----------|----------------------|----------------|
| "Tous les faits sur X" | Approximatif (top-K similaires) | Exact (tous les triplets de X) |
| "Lieux lies a X" | Mauvais (mots-cles "lieu" + "X") | Precis (relations typees) |
| Multi-sauts | Impossible | Natif (jointures SPARQL) |
| Filtrage par type | Limité | Natif (FILTER, rdf:type) |

> **Point cle** : SPARQL garantit une precision de 100% sur les faits du KG, alors que la recherche vectorielle ne fournit que des approximations.

### 3.3 Extraction de sous-graphe

Pour fournir un contexte au LLM, on extrait le **voisinage** d'une entite : tous les triplets directement lies a cette entite (1 saut) ou a 2 sauts.

In [ ]:
def extract_subgraph(graph, entity_uri, max_hops=2):
    """Extrait le sous-graphe autour d'une entite (N sauts)."""
    subgraph = Graph()
    subgraph.bind("ex", EX)
    subgraph.bind("rel", REL)
    subgraph.bind("rdfs", RDFS)

    visited = set()
    frontier = {entity_uri}

    for hop in range(max_hops):
        next_frontier = set()
        for entity in frontier:
            if entity in visited:
                continue
            visited.add(entity)

            # Triplets ou l'entite est sujet
            for s, p, o in graph.triples((entity, None, None)):
                subgraph.add((s, p, o))
                if isinstance(o, URIRef):
                    next_frontier.add(o)

            # Triplets ou l'entite est objet
            for s, p, o in graph.triples((None, None, entity)):
                subgraph.add((s, p, o))
                if isinstance(s, URIRef):
                    next_frontier.add(s)

        frontier = next_frontier - visited

    return subgraph


# Extraire le sous-graphe autour de Napoleon (2 sauts)
napoleon_uri = EX["Napoleon_Bonaparte"]
subgraph = extract_subgraph(kg, napoleon_uri, max_hops=2)

print(f"Sous-graphe autour de Napoleon Bonaparte :")
print(f"  Triplets (graphe complet) : {len(kg)}")
print(f"  Triplets (sous-graphe)    : {len(subgraph)}")
print(f"  Ratio                     : {len(subgraph)/len(kg)*100:.0f}%")
print(f"\nTriplets du sous-graphe :")
for s, p, o in sorted(subgraph, key=lambda x: str(x[0])):
    s_l = str(s).split('/')[-1].replace('_', ' ')
    p_l = str(p).split('/')[-1]
    o_l = str(o).split('/')[-1].replace('_', ' ') if isinstance(o, URIRef) else str(o)
    print(f"  ({s_l}, {p_l}, {o_l})")

### Interpretation : sous-graphe extrait

Le sous-graphe a 2 sauts capture l'essentiel des informations liees a Napoleon, y compris les details sur les entites connectees (ex: Sainte-Helene est une ile britannique).

**Comparaison des profondeurs de sous-graphe** :

| Profondeur | Triplets | Information | Usage |
|-----------|----------|-------------|-------|
| 1 saut | ~15 | Faits directs | Questions simples |
| 2 sauts | ~30+ | Faits + contexte | Questions complexes |
| 3+ sauts | Tout le graphe | Trop de bruit | Rarement utile |

> **Bonne pratique** : 2 sauts est generalement le meilleur compromis entre completude et pertinence.

---

## 4. Pipeline GraphRAG simplifie

Assemblons maintenant les composants precedents en un **pipeline GraphRAG complet**. Le pipeline suit 4 etapes :
1. Detection d'entites dans la question utilisateur
2. Requete du KG pour recuperer le sous-graphe pertinent
3. Formatage du contexte KG en texte structure
4. Generation de la reponse par LLM (ou simulation)

### Etape 1 : Detection d'entites dans la question

Pour identifier quelles entites du KG sont mentionnees dans la question, nous utilisons un matching simple par labels RDFS.

In [ ]:
def detect_entities_in_question(question, graph):
    """Detecte les entites du KG mentionnees dans la question."""
    # Recuperer tous les labels du graphe
    entity_labels = {}
    for s, p, o in graph.triples((None, RDFS.label, None)):
        label = str(o).lower()
        entity_labels[label] = s

    # Chercher les entites dans la question (matching exact sur le label)
    question_lower = question.lower()
    found = []
    # Trier par longueur decroissante pour matcher les noms complets d'abord
    for label, uri in sorted(entity_labels.items(), key=lambda x: -len(x[0])):
        if label in question_lower:
            found.append({"label": label, "uri": uri})
            # Eviter les doublons partiels
            question_lower = question_lower.replace(label, "")

    return found


# Test avec plusieurs questions
test_questions = [
    "Ou est ne Napoleon Bonaparte ?",
    "Qui a vaincu Napoleon a Waterloo ?",
    "Qu'est-ce que le Code civil ?",
    "Ou Napoleon a-t-il ete exile ?"
]

for q in test_questions:
    entities = detect_entities_in_question(q, kg)
    entity_names = [e["label"] for e in entities]
    print(f"Q: {q}")
    print(f"   Entites detectees : {entity_names}\n")

### Interpretation : detection d'entites

La detection par matching de labels est simple mais efficace pour un KG de petite taille. En production, on utiliserait :
- **Entity linking** : desambiguisation vers des entites connues (Wikidata, DBpedia)
- **Fuzzy matching** : tolerance aux variantes orthographiques
- **NER + linking** : extraction d'entites puis alignement avec le KG

### Etape 2 : Requete du KG

Pour chaque entite detectee, on extrait le sous-graphe pertinent via SPARQL.

In [ ]:
def retrieve_kg_context(question, graph, max_hops=1):
    """Recupere le contexte KG pertinent pour une question."""
    entities = detect_entities_in_question(question, graph)

    if not entities:
        return [], "Aucune entite reconnue dans la question."

    # Extraire le sous-graphe pour chaque entite detectee
    all_triples = []
    for entity in entities:
        sub = extract_subgraph(graph, entity["uri"], max_hops=max_hops)
        for s, p, o in sub:
            # Ignorer les labels RDFS pour le contexte
            if p == RDFS.label:
                continue
            s_label = str(s).split('/')[-1].replace('_', ' ')
            p_label = str(p).split('/')[-1]
            if isinstance(o, URIRef):
                o_label = str(o).split('/')[-1].replace('_', ' ')
            else:
                o_label = str(o)
            triple_str = f"{s_label} -- {p_label} --> {o_label}"
            if triple_str not in all_triples:
                all_triples.append(triple_str)

    entity_names = [e["label"] for e in entities]
    summary = f"Entites : {', '.join(entity_names)} | Faits : {len(all_triples)}"

    return all_triples, summary


# Test
question = "Ou est ne Napoleon Bonaparte et ou a-t-il ete exile ?"
triples, summary = retrieve_kg_context(question, kg, max_hops=1)

print(f"Question : {question}")
print(f"Resume   : {summary}")
print(f"\nFaits retrouves du KG :")
for t in triples:
    print(f"  {t}")

### Etape 3 : Formatage du contexte KG

Le contexte KG doit etre formate en texte structure pour etre injecte dans le prompt du LLM.

In [ ]:
def format_kg_context(triples, question):
    """Formate le contexte KG en texte structure pour le LLM."""
    if not triples:
        return "Aucun fait pertinent trouve dans le graphe de connaissances."

    context = "FAITS STRUCTURES (extraits du graphe de connaissances) :\n"
    context += "-" * 50 + "\n"

    # Grouper par sujet
    by_subject = defaultdict(list)
    for t in triples:
        parts = t.split(" -- ")
        if len(parts) >= 2:
            subject = parts[0].strip()
            rest = " -- ".join(parts[1:])
            by_subject[subject].append(rest)

    for subject, facts in by_subject.items():
        context += f"\n{subject} :\n"
        for fact in facts:
            context += f"  - {fact}\n"

    return context


# Test
formatted = format_kg_context(triples, question)
print(formatted)

### Etape 4 : Generation de la reponse

Enfin, nous envoyons la question et le contexte KG au LLM. Si aucune cle API n'est disponible, une reponse simulee est produite.

In [ ]:
def generate_response(question, kg_context):
    """Genere une reponse en utilisant le LLM avec le contexte KG."""
    system_prompt = """Tu es un assistant qui repond aux questions en te basant
UNIQUEMENT sur les faits fournis dans le contexte structure.
Si le contexte ne contient pas l'information, dis-le clairement.
Cite les faits utilises dans ta reponse."""

    user_prompt = f"""Contexte :
{kg_context}

Question : {question}

Reponds en francais en citant les faits du graphe de connaissances."""

    try:
        if OPENAI_KEY:
            from openai import OpenAI
            client = OpenAI(api_key=OPENAI_KEY)
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0
            )
            return response.choices[0].message.content, "openai"

        elif ANTHROPIC_KEY:
            from anthropic import Anthropic
            client = Anthropic(api_key=ANTHROPIC_KEY)
            response = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=1000,
                system=system_prompt,
                messages=[{"role": "user", "content": user_prompt}]
            )
            return response.content[0].text, "anthropic"

        else:
            return None, None

    except Exception as e:
        print(f"Erreur LLM : {e}")
        return None, None


# Reponse simulee
SIMULATED_RESPONSE = """D'apres le graphe de connaissances :

Napoleon Bonaparte est ne a **Ajaccio**, en Corse (fait : born_in_place -> Ajaccio,
Ajaccio located_in -> Corse).

Apres sa defaite a Waterloo en 1815, il a ete exile a **Sainte-Helene**,
une ile britannique situee dans l'Atlantique Sud (faits : exiled_to -> Sainte-Helene,
Sainte-Helene type -> Ile britannique, Sainte-Helene located_in -> Atlantique Sud).

Il y est mort en 1821 (fait : died_year -> 1821)."""

# Execution du pipeline complet
question = "Ou est ne Napoleon Bonaparte et ou a-t-il ete exile ?"
print(f"Question : {question}")
print("=" * 60)

# Etapes 1-3
triples, summary = retrieve_kg_context(question, kg, max_hops=2)
kg_context = format_kg_context(triples, question)
print(f"\n[Retrieval] {summary}")

# Etape 4
response, provider = generate_response(question, kg_context)
if response:
    print(f"\n[Generation via {provider}]")
    print(response)
else:
    print(f"\n[Generation simulee - pas de cle API]")
    print(SIMULATED_RESPONSE)

### Interpretation : pipeline GraphRAG

Le pipeline GraphRAG complet fonctionne en 4 etapes orchestrees :

| Etape | Entree | Sortie | Composant |
|-------|--------|--------|-----------|
| 1. Detection | Question texte | Liste d'entites | Matching labels KG |
| 2. Retrieval | Entites | Sous-graphe | SPARQL / traversee |
| 3. Formatage | Triplets | Texte structure | Template |
| 4. Generation | Contexte + question | Reponse | LLM |

**Avantage cle** : la reponse est **fondee sur des faits verifiables** du KG. Chaque assertion peut etre tracee jusqu'a un triplet specifique, ce qui reduit drastiquement les hallucinations.

> **Note technique** : En production, l'etape 4 combine souvent le contexte KG avec des passages textuels retrouves par recherche vectorielle (approche hybride).

### Pipeline complet : classe GraphRAG

Encapsulons le pipeline dans une classe reutilisable.

In [ ]:
class SimpleGraphRAG:
    """Pipeline GraphRAG simplifie."""

    def __init__(self, knowledge_graph, max_hops=2):
        self.kg = knowledge_graph
        self.max_hops = max_hops

    def query(self, question):
        """Execute le pipeline complet."""
        # Etape 1 : Detection d'entites
        entities = detect_entities_in_question(question, self.kg)

        # Etape 2 : Retrieval KG
        triples, summary = retrieve_kg_context(question, self.kg, self.max_hops)

        # Etape 3 : Formatage
        context = format_kg_context(triples, question)

        # Etape 4 : Generation
        response, provider = generate_response(question, context)

        return {
            "question": question,
            "entities_found": [e["label"] for e in entities],
            "kg_facts": len(triples),
            "context": context,
            "response": response,
            "provider": provider,
            "grounded": response is not None
        }


# Instancier le pipeline
rag = SimpleGraphRAG(kg, max_hops=2)

# Tester avec plusieurs questions
questions = [
    "Qui a epouse Napoleon Bonaparte ?",
    "Quand a eu lieu la bataille de Waterloo ?",
    "Quel est l'heritage de Napoleon ?"
]

for q in questions:
    result = rag.query(q)
    print(f"Q: {q}")
    print(f"   Entites   : {result['entities_found']}")
    print(f"   Faits KG  : {result['kg_facts']}")
    if result["response"]:
        # Afficher les premieres lignes
        lines = result["response"].split('\n')[:3]
        print(f"   Reponse   : {lines[0]}")
    else:
        print(f"   Reponse   : [simulee - pas de cle API]")
    print()

### Interpretation : classe SimpleGraphRAG

La classe `SimpleGraphRAG` encapsule les 4 etapes du pipeline. Pour chaque question, elle retourne un dictionnaire avec toutes les informations de tracing :

| Champ | Description | Utilite |
|-------|-------------|----------|
| `entities_found` | Entites detectees | Debugging du retrieval |
| `kg_facts` | Nombre de faits KG | Evaluation de la couverture |
| `context` | Texte structure | Transparence du prompt |
| `response` | Reponse generee | Resultat final |
| `grounded` | True si LLM utilise | Distinction simule/reel |

**Points cles** :
1. Le pipeline est **modulaire** : chaque etape peut etre amelioree independamment
2. Le **tracing** permet de verifier quels faits ont ete utilises
3. Le mode degrade (sans API) reste informatif grace au contexte KG

---

## 5. Ontologies dans le RAG

Les ontologies (OWL, RDFS) jouent un role crucial dans GraphRAG en apportant de la **semantique formelle** au graphe de connaissances.

### 5.1 Desambiguisation des requetes

Les ontologies definissent des hierarchies de classes et de proprietes qui aident a comprendre le sens des termes. Par exemple :

```turtle
# Hierarchie de classes
ex:Personne rdf:type owl:Class .
ex:Dirigeant rdfs:subClassOf ex:Personne .
ex:Empereur rdfs:subClassOf ex:Dirigeant .
ex:Napoleon rdf:type ex:Empereur .

# Hierarchie de proprietes
ex:ne_a rdfs:domain ex:Personne ;
        rdfs:range ex:Lieu .
```

**Avantages pour le retrieval** :

| Capacite | Sans ontologie | Avec ontologie |
|----------|---------------|----------------|
| "Tous les dirigeants" | Uniquement ceux tagges "dirigeant" | Empereurs, rois, presidents (par heritage) |
| "Lieu de naissance" | Matching exact "ne_a" | Aussi "born_in", "birthPlace" (equivalences) |
| Validation | Aucune | "ne_a" attend un Lieu, pas une Date |

In [ ]:
# Enrichir le KG avec une mini-ontologie
HIST = Namespace("http://example.org/ontology/history#")
kg.bind("hist", HIST)

# Classes
kg.add((HIST.Personne, RDF.type, OWL.Class))
kg.add((HIST.Dirigeant, RDF.type, OWL.Class))
kg.add((HIST.Dirigeant, RDFS.subClassOf, HIST.Personne))
kg.add((HIST.Empereur, RDF.type, OWL.Class))
kg.add((HIST.Empereur, RDFS.subClassOf, HIST.Dirigeant))
kg.add((HIST.Lieu, RDF.type, OWL.Class))
kg.add((HIST.Evenement, RDF.type, OWL.Class))
kg.add((HIST.Institution, RDF.type, OWL.Class))

# Typage des instances
kg.add((EX.Napoleon_Bonaparte, RDF.type, HIST.Empereur))
kg.add((EX.Ajaccio, RDF.type, HIST.Lieu))
kg.add((EX.Corse, RDF.type, HIST.Lieu))
kg.add((EX.Waterloo, RDF.type, HIST.Lieu))
kg.add((EX["Sainte-Helene"], RDF.type, HIST.Lieu))
kg.add((EX.Banque_de_France, RDF.type, HIST.Institution))
kg.add((EX.Bataille_de_Waterloo, RDF.type, HIST.Evenement))

# Proprietes avec domaine et range
kg.add((REL.born_in_place, RDFS.domain, HIST.Personne))
kg.add((REL.born_in_place, RDFS.range, HIST.Lieu))
kg.add((REL.defeated_at, RDFS.domain, HIST.Personne))
kg.add((REL.defeated_at, RDFS.range, HIST.Lieu))

# Requete : trouver tous les lieux (par type)
query_lieux = """
PREFIX hist: <http://example.org/ontology/history#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?lieu ?label
WHERE {
    ?lieu a hist:Lieu .
    OPTIONAL { ?lieu rdfs:label ?label }
}
"""

print("Lieux identifies par type ontologique :")
for row in kg.query(query_lieux):
    name = str(row.label) if row.label else str(row.lieu).split('/')[-1].replace('_', ' ')
    print(f"  - {name}")

# Requete : inference via rdfs:subClassOf
print("\nHierarchie de classes :")
print("  Empereur --> subClassOf --> Dirigeant --> subClassOf --> Personne")
print(f"  Napoleon est un Empereur : type direct")
print(f"  Napoleon est un Dirigeant : par inference (subClassOf)")
print(f"  Napoleon est une Personne : par inference transitive")

### Interpretation : ontologies et retrieval

L'ajout d'une ontologie au KG apporte trois benefices majeurs pour le retrieval :

| Benefice | Mecanisme | Exemple |
|----------|-----------|----------|
| **Categorisation** | `rdf:type` + `rdfs:subClassOf` | Trouver "tous les lieux" par type |
| **Desambiguisation** | `rdfs:domain` / `rdfs:range` | "ne_a" attend un Lieu, pas une Date |
| **Inference** | Heritage de classes | Un Empereur EST un Dirigeant |

> **Point cle** : Les ontologies permettent au systeme GraphRAG de repondre a des questions que l'utilisateur n'a pas formulees exactement selon les termes du KG.

### 5.2 Validation des sorties LLM

Le KG peut aussi servir a **valider** les affirmations d'un LLM apres generation, en verifiant si elles sont coherentes avec les faits connus.

In [ ]:
def validate_claim(graph, subject_name, predicate_name, object_value):
    """Verifie si une affirmation est coherente avec le KG."""
    subject_uri = name_to_uri(subject_name)

    # Chercher les triplets correspondants
    for s, p, o in graph.triples((subject_uri, None, None)):
        p_label = str(p).split('/')[-1].lower()
        o_label = str(o).split('/')[-1].replace('_', ' ').lower() if isinstance(o, URIRef) else str(o).lower()

        if predicate_name.lower() in p_label:
            if object_value.lower() in o_label or o_label in object_value.lower():
                return "CONFIRME", f"Triplet trouve : ({subject_name}, {p_label}, {o_label})"
            else:
                return "CONTREDIT", f"KG dit : {o_label} (pas {object_value})"

    return "NON VERIFIE", "Aucun triplet correspondant dans le KG"


# Tester des affirmations
claims = [
    ("Napoleon Bonaparte", "born_in", "Ajaccio"),      # Vrai
    ("Napoleon Bonaparte", "born_in", "Paris"),          # Faux
    ("Napoleon Bonaparte", "died", "1821"),              # Vrai
    ("Napoleon Bonaparte", "allied_with", "Angleterre"), # Non verifiable
]

print("Validation d'affirmations contre le KG :")
print("=" * 65)
for subj, pred, obj in claims:
    status, detail = validate_claim(kg, subj, pred, obj)
    print(f"  '{subj} {pred} {obj}'")
    print(f"    -> {status} : {detail}\n")

### Interpretation : validation factuelle

La validation par KG produit trois resultats possibles :

| Statut | Signification | Action |
|--------|---------------|--------|
| **CONFIRME** | Le fait existe dans le KG | Confiance elevee |
| **CONTREDIT** | Le KG contient un fait different | Alerte hallucination |
| **NON VERIFIE** | Le KG n'a pas d'information | Confiance moderee |

**Points cles** :
1. La validation post-generation reduit les hallucinations factuelles
2. Les faits **CONTREDITS** sont les plus precieux (detection d'erreurs)
3. Les faits **NON VERIFIES** necessitent d'autres sources de verification

> **Note technique** : Cette approche est utilisee en production par des systemes comme Babelscape REBEL et Meta KILT pour le fact-checking automatise.

---

## 6. Evaluation et limitations

### 6.1 Reduction des hallucinations

L'avantage principal de GraphRAG est la **reduction des hallucinations** grace au grounding factuel.

In [ ]:
# Simulation d'evaluation : comparaison RAG vs GraphRAG
evaluation_results = {
    "questions": [
        "Ou est ne Napoleon ?",
        "Qui Napoleon a-t-il epouse ?",
        "Quand a eu lieu Waterloo ?",
        "Quel est l'heritage de Napoleon ?",
        "Quels lieux sont lies a Napoleon ?"
    ],
    "rag_classic": {
        "correct": [True, True, True, True, False],
        "hallucinations": [False, False, False, True, True],
        "complete": [True, False, True, False, False]  # RAG manque des details
    },
    "graphrag": {
        "correct": [True, True, True, True, True],
        "hallucinations": [False, False, False, False, False],
        "complete": [True, True, True, True, True]
    }
}

n = len(evaluation_results["questions"])
rag_accuracy = sum(evaluation_results["rag_classic"]["correct"]) / n
rag_halluc = sum(evaluation_results["rag_classic"]["hallucinations"]) / n
rag_complete = sum(evaluation_results["rag_classic"]["complete"]) / n

grag_accuracy = sum(evaluation_results["graphrag"]["correct"]) / n
grag_halluc = sum(evaluation_results["graphrag"]["hallucinations"]) / n
grag_complete = sum(evaluation_results["graphrag"]["complete"]) / n

print("Evaluation comparative (donnees simulees)")
print("=" * 55)
print(f"{'Metrique':<25} {'RAG classique':>15} {'GraphRAG':>15}")
print("-" * 55)
print(f"{'Precision':<25} {rag_accuracy:>14.0%} {grag_accuracy:>14.0%}")
print(f"{'Hallucinations':<25} {rag_halluc:>14.0%} {grag_halluc:>14.0%}")
print(f"{'Completude':<25} {rag_complete:>14.0%} {grag_complete:>14.0%}")
print(f"\nNote : Ces resultats illustrent les tendances observees")
print(f"dans la litterature (Microsoft GraphRAG 2024, etc.).")
print(f"Les chiffres exacts varient selon le domaine et le corpus.")

### Interpretation : evaluation comparative

Les resultats (simules ici pour illustration) refletent les tendances documentees dans la litterature :

| Metrique | RAG classique | GraphRAG | Explication |
|----------|--------------|----------|-------------|
| Precision | ~80% | ~100% | KG fournit des faits verifies |
| Hallucinations | ~40% | ~0% | Grounding factuel |
| Completude | ~60% | ~100% | Traversee de graphe multi-sauts |

> **Reference** : L'article original de Microsoft (Edge et al., 2024) rapporte une amelioration de 20-30% sur les questions globales par rapport au RAG classique.

### 6.2 Analyse des couts

GraphRAG introduit un cout supplementaire de construction et maintenance du KG.

In [ ]:
# Estimation des couts (ordres de grandeur)
cost_analysis = [
    ("Construction de l'index", "Embedding vectoriel", "Extraction KG par LLM"),
    ("Cout par document (1000 tokens)", "~$0.001 (embedding)", "~$0.01-0.05 (extraction LLM)"),
    ("Cout par requete", "~$0.001 (embedding query)", "~$0.005 (SPARQL + embedding)"),
    ("Stockage", "Vecteurs (~1KB/doc)", "Triplets (~0.5KB/doc) + index"),
    ("Temps d'indexation", "Secondes (embedding)", "Minutes (extraction LLM)"),
    ("Temps de requete", "~100ms (ANN search)", "~200ms (SPARQL + ANN)"),
    ("Mise a jour", "Re-embed le document", "Re-extraire les entites"),
]

print("Analyse des couts : RAG classique vs GraphRAG")
print("=" * 75)
print(f"{'Dimension':<30} {'RAG classique':<22} {'GraphRAG'}")
print("-" * 75)
for dimension, rag, graphrag in cost_analysis:
    print(f"{dimension:<30} {rag:<22} {graphrag}")

print(f"\nConclusion : GraphRAG coute 5-50x plus cher a construire,")
print(f"mais produit des reponses plus fiables et structurees.")

### 6.3 Quand utiliser GraphRAG vs RAG classique ?

| Critere | Privilegier RAG classique | Privilegier GraphRAG |
|---------|--------------------------|----------------------|
| Volume de donnees | Grand corpus (>100K docs) | Corpus moyen (<10K docs) |
| Nature des questions | Questions locales ("de quoi parle ce document ?") | Questions globales ou relationnelles |
| Besoin de precision | Tolerant aux approximations | Exigence de faits verifiables |
| Budget | Limite | Disponible |
| Frequence de mise a jour | Elevee (donnees volatiles) | Moderee (donnees stables) |
| Domaine | General, ouvert | Specialise (medical, juridique, technique) |
| Relations | Peu importantes | Centrales (reseaux, genealogies, supply chain) |

> **Recommandation pratique** : Pour la plupart des cas d'usage, une approche **hybride** (RAG vectoriel + KG) offre le meilleur compromis. On utilise le KG pour les faits critiques et la recherche vectorielle pour le contexte general.

---

## 7. Directions futures

Le domaine GraphRAG evolue rapidement. Voici les tendances majeures pour 2025-2027.

### 7.1 Convergence neuro-symbolique

L'IA **neuro-symbolique** combine les forces des reseaux de neurones (apprentissage, generalisation) avec celles des systemes symboliques (raisonnement, explicabilite) :

```
Approche neuronale pure        Approche hybride           Approche symbolique pure
  (LLM, embeddings)      (GraphRAG, neuro-symbolique)    (logique, ontologies)
       |                          |                              |
  Apprentissage             Apprentissage +                Raisonnement
  statistique               raisonnement                   formel
       |                          |                              |
  + Generalisation          + Precision                    + Garanties
  + Flexibilite             + Explicabilite                + Explicabilite
  - Hallucinations          + Reduction                    - Fragilite
  - Opacite                   hallucinations               - Couverture limitee
```

GraphRAG est une forme concrete de cette convergence : le KG apporte la structure symbolique, le LLM apporte la flexibilite neuronale.

### 7.2 KG Embeddings

Les **KG Embeddings** representent les entites et relations d'un graphe dans un espace vectoriel continu, permettant de combiner le meilleur des deux mondes :

| Methode | Principe | Avantage |
|---------|----------|----------|
| **TransE** (2013) | Relation = translation vectorielle : h + r = t | Simple, rapide |
| **RotatE** (2019) | Relation = rotation dans l'espace complexe | Capture les patterns (symetrie, inversion) |
| **CompGCN** (2020) | GNN sur le KG, compositions de relations | Raisonnement multi-sauts |
| **NodePiece** (2022) | Tokenisation des noeuds, transferable | Scalable, inductif |

**Principe de TransE** :

Pour un triplet (h, r, t), le modele apprend des vecteurs tels que :

$$\vec{h} + \vec{r} \approx \vec{t}$$

Exemple : vec(Paris) + vec(capitale_de) = vec(France)

**Application au GraphRAG** : les embeddings de KG permettent de retrouver des entites similaires meme si elles ne sont pas directement liees dans le graphe, combinant precision structurelle et generalisation.

### 7.3 Agentic RAG avec outils KG

Les **agents IA** (systemes autonomes bases sur LLM) peuvent utiliser le KG comme un **outil** parmi d'autres :

```
Agent IA
  |
  |-- Outil : Recherche vectorielle (RAG classique)
  |-- Outil : Requete SPARQL (GraphRAG)
  |-- Outil : Validation factuelle (KG check)
  |-- Outil : Calcul (Python/Wolfram)
  |-- Outil : Navigation web
  |
  v
L'agent choisit dynamiquement quel outil utiliser
selon la question posee.
```

Cette approche est plus flexible qu'un pipeline fixe : l'agent decide quand interroger le KG vs faire une recherche vectorielle vs valider un fait.

### 7.4 Tendances 2025-2027

| Tendance | Horizon | Impact | Maturite |
|----------|---------|--------|----------|
| GraphRAG open-source (LlamaIndex, LangChain) | 2025 | Democratisation | Production |
| KG Embeddings + LLMs | 2025-2026 | Retrieval hybride | Recherche avancee |
| Agentic RAG avec KG comme outil | 2025-2026 | Flexibilite | Emerging |
| Construction automatique de KG (LLM-powered) | 2025 | Reduction cout construction | Production |
| KG temporels (faits avec validite) | 2026 | Precision temporelle | Recherche |
| KG multimodaux (texte + images + tables) | 2026-2027 | Couverture | Recherche |
| Federated GraphRAG (KGs distribues) | 2027 | Scale + vie privee | Conceptuel |

> **Point cle** : GraphRAG est au croisement de deux tendances majeures de l'IA : les **LLMs** (generation) et les **graphes de connaissances** (structure). Leur convergence est une direction de recherche tres active.

---

## Exercices

### Exercice 1 : Extraction d'entites par regles

Ecrivez une fonction `extract_entities_science(text)` qui extrait des entites (scientifiques, theories, institutions) a partir du texte suivant sur la physique moderne. Utilisez des expressions regulieres.

In [ ]:
PHYSICS_TEXT = """
Albert Einstein a publie la theorie de la relativite restreinte en 1905 a Berne.
Il a recu le prix Nobel de physique en 1921 pour l'effet photoelectrique.
Max Planck a introduit le quantum d'action en 1900 a Berlin.
Niels Bohr a propose son modele atomique en 1913 a Copenhague.
Werner Heisenberg a formule le principe d'incertitude en 1927.
"""

def extract_entities_science(text):
    """Exercice : extraire les entites scientifiques du texte."""
    entities = {
        "scientists": [],
        "theories": [],
        "dates": [],
        "places": []
    }

    # TODO : Implementez l'extraction
    # Indices :
    #   - Dates : re.findall(r'\b(\d{4})\b', text)
    #   - Scientifiques : patterns "Prenom Nom"
    #   - Theories : patterns "theorie de...", "principe de...", "modele..."
    #   - Lieux : patterns "a Ville"

    return entities


# Testez votre solution
# result = extract_entities_science(PHYSICS_TEXT)
# print(result)

### Exercice 2 : Mini KG et requete de contexte

Construisez un mini graphe RDF sur le systeme solaire (planetes, etoile, satellites) et ecrivez une fonction qui retourne le contexte KG pour la question "Quels sont les satellites de Jupiter ?".

In [ ]:
def build_solar_system_kg():
    """Exercice : construire un KG du systeme solaire."""
    g = Graph()
    ASTRO = Namespace("http://example.org/astro/")
    g.bind("astro", ASTRO)

    # TODO : Ajoutez des triplets pour :
    # - Le Soleil (type: Etoile)
    # - Les planetes (Terre, Jupiter, Mars...)
    #   avec relation "orbite_autour" -> Soleil
    # - Les satellites (Lune, Europe, Io, Ganymede, Callisto...)
    #   avec relation "orbite_autour" -> leur planete
    # - Des proprietes : diametre, distance_au_soleil, etc.

    return g


def query_satellites(graph, planet_name):
    """Exercice : requete SPARQL pour les satellites d'une planete."""
    # TODO : Ecrivez une requete SPARQL qui retourne
    # tous les satellites orbitant autour de planet_name
    pass


# Testez votre solution
# kg_solar = build_solar_system_kg()
# satellites = query_satellites(kg_solar, "Jupiter")
# print(f"Satellites de Jupiter : {satellites}")

### Exercice 3 : Pipeline GraphRAG pour un domaine specifique

Concevez un pipeline GraphRAG pour le domaine **medical** (maladies, symptomes, traitements). Definissez :
1. Le schema du KG (classes, proprietes)
2. Des exemples de triplets
3. Des exemples de questions et le sous-graphe attendu

In [ ]:
def design_medical_graphrag():
    """Exercice : concevoir un pipeline GraphRAG medical."""

    # TODO : Definissez le schema (classes et proprietes)
    schema = {
        "classes": [
            # ex: "Maladie", "Symptome", "Traitement", "Medicament", "Organe"
        ],
        "properties": [
            # ex: ("a_symptome", "Maladie", "Symptome"),
            #     ("traite_par", "Maladie", "Traitement"),
        ]
    }

    # TODO : Exemples de triplets
    example_triples = [
        # ex: ("Grippe", "a_symptome", "Fievre"),
        #     ("Grippe", "traite_par", "Paracetamol"),
    ]

    # TODO : Exemples de questions et sous-graphes attendus
    example_queries = [
        # ex: {
        #     "question": "Quels sont les symptomes de la grippe ?",
        #     "expected_subgraph": [...],
        # }
    ]

    return schema, example_triples, example_queries


# Testez votre conception
# schema, triples, queries = design_medical_graphrag()
# print(f"Classes : {schema['classes']}")
# print(f"Triplets : {len(triples)}")
# print(f"Questions : {len(queries)}")

---

## Conclusion de la serie Web Semantique

Ce notebook conclut la **serie de 13 notebooks** sur le Web Semantique. Voici un recapitulatif du parcours complet.

### Parcours d'apprentissage

| # | Notebook | Theme | Competences acquises |
|---|----------|-------|---------------------|
| 1 | SW-1-Setup | Installation | Environnement dotNetRDF, premier graphe |
| 2 | SW-2-RDFBasics | Fondations RDF | Triplets, noeuds, serialisation |
| 3 | SW-3-GraphOperations | Manipulation | Lecture, ecriture, fusion, selection |
| 4 | SW-4-SPARQL | Requetes | SELECT, FILTER, OPTIONAL, UNION |
| 5 | SW-5-LinkedData | Donnees liees | DBpedia, Wikidata, SERVICE |
| 6 | SW-6-RDFS | Schema | Classes, proprietes, inference |
| 7 | SW-7-OWL | Ontologies | OWL 2, raisonnement, profils |
| 8 | SW-8-PythonRDF | Python RDF | rdflib, SPARQLWrapper, transition .NET -> Python |
| 9 | SW-9-SHACL | Validation | Shapes, contraintes, qualite |
| 10 | SW-10-JSONLD | JSON-LD | Schema.org, donnees web structurees |
| 11 | SW-11-RDFStar | RDF 1.2 | Quoted triples, SPARQL-Star |
| 12 | SW-12-KnowledgeGraphs | Graphes de connaissances | Construction, kglab, visualisation |
| 13 | **SW-13-GraphRAG** | **KG + LLMs** | **Pipeline GraphRAG, extraction, validation** |

### Progression thematique

```
Partie 1 (SW-1 a 4) : FONDATIONS
  RDF -> Triplets -> Graphes -> SPARQL

Partie 2 (SW-5 a 7) : SEMANTIQUE
  Linked Data -> RDFS -> OWL

Partie 3 (SW-8 a 11) : ECOSYSTEME MODERNE
  Python -> SHACL -> JSON-LD -> RDF 1.2

Partie 4 (SW-12 a 13) : INTELLIGENCE ARTIFICIELLE
  Knowledge Graphs -> GraphRAG (KG + LLMs)
```

### Competences transversales

| Competence | Notebooks | Niveau atteint |
|-----------|-----------|----------------|
| Modelisation de donnees en graphe | 1-3, 8, 12 | Avance |
| Interrogation SPARQL | 4-5, 8, 13 | Avance |
| Ontologies et inference | 6-7 | Intermediaire |
| Validation et qualite | 9 | Intermediaire |
| Integration Web | 10-11 | Intermediaire |
| IA et graphes de connaissances | 12-13 | Introduction |

### Pour aller plus loin

**Articles de reference** :
- Edge et al. (2024), *"From Local to Global: A Graph RAG Approach to Query-Focused Summarization"*, Microsoft Research
- Bordes et al. (2013), *"Translating Embeddings for Modeling Multi-relational Data"* (TransE)
- Sun et al. (2019), *"RotatE: Knowledge Graph Embedding by Relational Rotation"*
- Pan et al. (2024), *"Unifying Large Language Models and Knowledge Graphs: A Roadmap"*

**Outils et frameworks** :
- [Microsoft GraphRAG](https://github.com/microsoft/graphrag) : implementation open-source
- [LlamaIndex Knowledge Graph](https://docs.llamaindex.ai/) : integration KG dans LlamaIndex
- [LangChain Graph](https://python.langchain.com/) : GraphRAG dans LangChain
- [Neo4j + LLM](https://neo4j.com/labs/genai-ecosystem/) : graphes de proprietes + IA generative

**Standards W3C couverts dans la serie** :
- RDF 1.1 / 1.2, SPARQL 1.1, RDFS 1.0, OWL 2, SHACL 1.0, JSON-LD 1.1

---

## Resume

| Element | Ce que nous avons appris |
|---------|-------------------------|
| RAG vs GraphRAG | GraphRAG ajoute un KG structure au pipeline RAG classique |
| Extraction d'entites | Par regles (deterministe) ou par LLM (flexible) |
| Construction de KG | Texte -> triplets -> graphe RDF avec rdflib |
| KG-enhanced retrieval | SPARQL + sous-graphe pour un contexte structure |
| Pipeline GraphRAG | Detection -> Requete KG -> Formatage -> Generation LLM |
| Ontologies et RAG | Desambiguisation, inference, validation factuelle |
| Evaluation | Reduction hallucinations, precision accrue, cout plus eleve |
| Directions futures | Neuro-symbolique, KG embeddings, Agentic RAG |

---

**Navigation** : [<< 12-KnowledgeGraphs](SW-12-KnowledgeGraphs.ipynb) | [Index](README.md)